In [19]:
import qiskit
import numpy as np
from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.noise import NoiseModel, depolarizing_error

In [20]:
from qiskit import QuantumCircuit

def circular_ansatz_mirrored(N, reps=1, fix_2q=False): 
    qc = qiskit.QuantumCircuit(N)
    for _ in range(reps):
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N-1, -1, -1):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
    for i in range(N):
        qc.ry(np.pi/2, i)
    for i in range(N):
        qc.rz(np.pi/2, i)
    return qc


def populated_circuit(n):
    qc = QuantumCircuit(n)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
        if i + 2 < n:
            qc.cz(i + 1, i + 2)
    return qc


In [ ]:
num_qubits = 20

paulis = ["".join(np.random.choice(['I', 'X', 'Y', 'Z'], size=num_qubits)) for _ in range(6)]

coeffs = np.random.random(len(paulis))


# qc = circular_ansatz_mirrored(num_qubits)
qc = populated_circuit(num_qubits)


In [22]:
backend = FakeMumbaiV2()
ideal_estimator = Estimator(mode=backend)

# Or define a real backend
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService()
# backend = service.least_busy(operational=True, simulator=False)

# estimator.options.twirling.enable_gates = True
# estimator.options.twirling.num_randomizations = 32
# estimator.options.twirling.shots_per_randomization = 100

In [23]:
np.random.seed(0)

# ZX means Z on the first qubit and X on the second qubit
coeffs = np.random.random(len(paulis))
observable = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

In [24]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
 
isa_observable = observable.apply_layout(isa_circuit.layout)

In [25]:
job = ideal_estimator.run([(isa_circuit, isa_observable)])

In [26]:
# Get results for the first (and only) PUB
 
ideal_res = job.result()[0]
print(f">>> Expectation value: {ideal_res.data.evs}")

>>> Expectation value: 0.00187555534809638


In [27]:
noisy_estimator = Estimator(mode=backend)


noise_model = NoiseModel()
cx_depolarizing_prob = 0.1
noise_model.add_all_qubit_quantum_error(
    depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
)

(noisy_estimator.options.simulator.noise_model) = noise_model

In [28]:
# Get results for the first (and only) PUB

job = noisy_estimator.run([(isa_circuit, isa_observable)])
noisy_res = job.result()[0]


print(f">>> Expectation value: {noisy_res.data.evs}")
 

>>> Expectation value: -0.03007811628572027


In [29]:
abs_error_diff = abs(noisy_res.data.evs - ideal_res.data.evs)
print(f"Absolute error difference: {abs_error_diff}")

Absolute error difference: 0.03195367163381665
